In [ ]:
# Questo script è un progetto di automazione volto a trovare nuovi Datasets pertinenti con la nostra
# analisi sul clima, cercando per parole chiave i Datasets su www.kaggle.com e importando le loro informazioni 
# su un foglio di Google Sheets, così da avere sempre nuovi dati da visionare e, se utili, da implementare.

# Il progetto per adesso funziona solo in locale e le chiavi API non sono protette da variabili d'ambiente,
# migliorie che verranno apportate, come l'esecuzione automatica di questo script (richiamandolo con un trigger
# temporale da AppScript ed eseguendosi così periodicamente in maniera automatica).

import os
from flask import Flask, jsonify
import kaggle
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import datetime
import gspread

app = Flask(__name__)


os.environ['KAGGLE_CONFIG_DIR'] = '/Users/matteobalducci/Documents/'
os.environ['KAGGLE_USERNAME'] = 'matteobalducciiii'
os.environ['KAGGLE_KEY'] = '82993cf0a51a3e5c4028c2afa86b95f0'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

def get_gsheets_credentials():
    creds = None
    if os.path.exists('/Users/matteobalducci/Documents/token.pickle'):
        with open('/Users/matteobalducci/Documents/token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/matteobalducci/Documents/client_secret_119442292180-k5876356789071c1dnbai08418c8e6uf.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=8080)  # Modifica qui
        with open('/Users/matteobalducci/Documents/token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

@app.route('/')
def search_kaggle_datasets():
    keywords = ["climate", "temperature", "pollution", "CO2"]
    datasets = []

    for keyword in keywords:
        search_results = kaggle.api.datasets_list(search=keyword)
        for dataset in search_results[:10]:
            datasets.append({
                "title": dataset["title"],
                "url": f"https://www.kaggle.com/{dataset['ref']}",
                "datetime": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

    # Accesso a Google Sheets
    gc = gspread.authorize(get_gsheets_credentials())
    sh = gc.open_by_key('1twWLXP-sEVmr_XETGOOerfq7sI4Ot0UN8LoecGqpf7k')
    
    # Se il foglio "loggg" non esiste, lo crea
    try:
        worksheet = sh.worksheet("loggg")
    except:
        worksheet = sh.add_worksheet(title="loggg", rows="100", cols="4")

    # Aggiorna le informazioni nel foglio di lavoro
    for dataset in datasets:
        worksheet.append_row([dataset["title"], dataset["url"], dataset["datetime"]])

    return jsonify(datasets)

if __name__ == '__main__':
    app.run(debug=True)
